# Physics 494/594

# t-SNE & Clustering


In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
π = np.pi

## Last Time

### [Notebook Link: 31_Autoencoders.ipynb](31_Autoencoders.ipynb)

- Introduced unsupervised learning, or discovering patterns and structure in unlabelled data.
- Discussed principal component analysis, which allowed for dimensional reduction by projecting onto components which capture the largest variance (signal) in the data.
- Studied MNIST data set, and identified some clusters, but without prior labelling, separation into clusters would be problematic.

## Today
- Curse of dimensionality.
- Clustering: group unlabeled data into clusters according to some similarity metric.
    - t-SNE: preserving local structures when projecting to low dimensions.
    - Identification of latent variables.
    - Algorithms provided by `sklearn`


## Curse of Dimensionality

In high dimensions, everthing lives on the edge!

Consider a space with $D \gg 1$ and generate uniform random numbers inside a hypercube of side length $d$, i.e. $x \in \mathcal{U}_{\Omega}$ where $\Omega = d^D$.  Now let's determine how many of them fall within the hypersphere of diameter $d$ centered at the origin. Let's visualize for $D = 3$.

<!--
N = 2000
x = np.random.normal(loc=[0,0],scale=[1,0.4], size=[N,2])

θ = np.radians(35)
R = np.array([[np.cos(θ), -np.sin(θ)], [np.sin(θ), np.sin(θ)]])
for i in range(N):
    x[i] = R @ x[i]
np.savetxt('../data/scatter_2d_pca.dat',x)
-->

In [ ]:
N = 10**4
d = 1.0
D = 3
C = np.random.uniform(low=-d/2, high=d/2,size=(N,D))

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=plt.figaspect(1)*2)
ax = plt.axes(projection='3d')

# we find the points inside the hypersphere
S = np.where(np.sum(C**2,axis=1) < (d/2)**2)[0]

ax.scatter(C[S,0], C[S,1], C[S,2], c=colors[0], marker='.', rasterized=True)
ax.scatter(C[~S,0], C[~S,1], C[~S,2], c=colors[-2], marker='.', alpha=0.2, rasterized=True)

ax.set_xlabel('x',labelpad=14)
ax.set_ylabel('y',labelpad=14)
ax.set_zlabel('z',)

ax.set_xlim(-0.6,0.6)
ax.set_ylim(-0.6,0.6)
ax.set_zlim(-0.6,0.6)

print(f'Sphere/Cube = {S.shape[0]/C.shape[0]}')

However, as the dimension increases, this fraction is reduced rapidly and is given by the volume ratio:

\begin{equation}
\frac{V_{\mathcal{S}}}{\Omega} = \frac{\frac{\pi^{D/2} (d/2)^D}{\Gamma(D/2 + 1)}}{d^D} 
%\approx \frac{e^{-\frac{1}{2} D \left(\log \left(\frac{2 D}{\pi }\right)-1\right)}}{\sqrt{\pi } \sqrt{D}} 
\sim \left(\frac{1}{2}\right)^D\, .
\end{equation}

This tells us that for uniformly distributed data, the number of points found inside the hypersphere vanishes exponentially fast!

In [ ]:
%%time
N = 10**6
D = np.linspace(1,20,20).astype(int)

ratio = []
for cD in D:
    C = np.random.uniform(low=-d/2, high=d/2,size=(N,cD))
    S = np.where(np.sum(C**2,axis=1) < (d/2)**2)[0]
    ratio.append(S.shape[0]/N)

In [ ]:
from scipy.special import gamma as Γ
Vratio = np.exp(-0.5*D*(-1.0+np.log(2.0*D/π)))/(π*np.sqrt(D))
plt.semilogy(D,ratio,'o', mfc='None', color=colors[0], label='Data')
plt.semilogy(D,π**(D/2)*(d/2)**D/Γ(D/2+1),'-', color=colors[0], label=r'$V_\mathcal{S}/\Omega$')
plt.semilogy(D,0.5**D,'-', color=colors[3], label=r'$2^{-D}$', zorder=-1)
plt.xlabel('Spatial Dimension  (D)')
plt.ylabel('Edge to Bulk Ratio')
plt.legend();

### Real Data is non-Uniform

As we saw last time, we were able to understand most of the features of our $D=784$ MNIST data set with only a few principal components.  This is due to the large amount of *structure* in real-world data. There is a famous example called the *swiss roll* which demonstrates that the minimum number of parameters required to parameterize the data is equal to its intrinsic dimensionality.

In [ ]:
N = 5000
x1 = np.linspace(0,2,N)
x2 = np.random.uniform(low=0,high=4*π, size=N)

x = np.column_stack((x1,x2*np.sin(x2),x2*np.cos(x2)))

# extract colors corresponding to angle mod 2π
c = np.array(colors)[np.array(np.fmod(x2,2*π) // (2*π/10)).astype(int)]

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = [fig.add_subplot(1, 2, 1, projection='3d'),fig.add_subplot(1, 2, 2)]

ax[0].scatter(x[:,0],x[:,1],x[:,2], c=c, marker='.')

ax[0].set_xlabel('x',labelpad=14)
ax[0].set_ylabel('y',labelpad=14)
ax[0].set_zlabel('z')

ax[0].view_init(15, 30)

ax[1].scatter(x1,x2,c=c, marker='.')
ax[1].set_xlabel(r'$x_1$')
ax[1].set_ylabel(r'$x_2$');

### Can PCA determine the intrinsic dimensionality?

In [ ]:
from sklearn.decomposition import PCA

# perform the PCA
model = PCA(n_components=2)
XPCA = model.fit_transform(x)

# store the results
λ = model.explained_variance_
PCAj = model.explained_variance_ratio_
V = model.components_

# project onto the first 2 principal components
pX = x @ V.T

# visualize the results

plt.scatter(pX[:,0],pX[:,1], c=c, marker='.')

This is a simple projection onto the y-z plane (the two dimensions with greatest variance) so PCA is doing exaclty what it is supposed to, but it is not enough to fully identify patterns in the data.  This is related to the crowding problem that can arise when we try to preserve distances upon projection.

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,6))

square = np.array([[-0.5,-0.5],[-0.5,0.5],[0.5,-0.5],[0.5,0.5]])
ax[0].scatter(square[:,0],square[:,1], c=colors[:8:2], s=400)
ax[0].set_xticks([])
ax[0].set_yticks([]);

proj_square = np.array([[-0.5,0.500],[-0.5+0.01,0.5],[0.5,0.5],[0.5+0.01,0.5]])
ax[1].scatter(proj_square[:,0],proj_square[:,1], c=colors[:8:2], s=400)

ax[1].axhline(y=0, ls='--', color='k')

proj_square = np.array([[-np.sqrt(2)/2,-0.5],[-0.005,-0.5],[0.005,-0.5],[np.sqrt(2)/2,-0.5]])
ax[1].scatter(proj_square[:,0],proj_square[:,1], c=colors[:8:2], s=400)

ax[1].set_ylim(-1,1)
ax[1].axis('off');

This problem arises as we are trying to satisfy too many constraints (i.e. all distances preserved).  This can be addressed by weakening the constraint and prioritizing preserving distances that are **close** in the high-dimensional space.  

## t-distributed Stochastic Neighbor Embedding (t-SNE)

This is an extremely powerful algorithm developped in 2008:
- [L.J.P. van der Maaten and G.E. Hinton, *Visualizing High-Dimensional Data Using t-SNE*, Journal of Machine Learning Research **9**, 2579 (2008).](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)

The goal is to reproduce distances from the higher dimensional space inside the latent space as closely as possible, realizing this is not a perfectly possible (hence stochastic).

We can define a cost function $\mathcal{C}$ of the relative separation of points $\|\boldsymbol{x}_i-\boldsymbol{x}_j\|$ in the $D$ dimensional space and $\|\boldsymbol{y}_i-\boldsymbol{y}_j\|$ in the latent lower dimensional space.

\begin{equation}
\mathcal{C} = \sum_{i \ne j} F(\|\boldsymbol{x}_i-\boldsymbol{x}_j\|,\|\boldsymbol{y}_i-\boldsymbol{y}_j\|)\, ,
\end{equation}

that can be minimized using our tools like gradient descent. We want to choose a function that imposes a penalty (repulsion) if points in the low-dimensional space are closer than in the higher dimensional one, and otherwise attracts them.  In *Stochastic Neighbor Embedding*, this is accomplished by defining probability distributions associated with the neighborhood of each data point in the high-dimensional ($p_{ij}$) and low-dimensional ($q_{ij}$) spaces subject to the normalization condition that $\sum_{i\ne j} p_{ij} = \sum_{i\ne j} q_{ij} = 1$.

We want to pick these distributions such that the probability to pick a pair of points is **larger** if they are close neighbors in the high-dimensional space, and similar in the low-dimenisional one.  Our goal is to make these two probability distributions as close as possible (within the restrictions of embedding to the latent space).   This is a well-studied problem in probability theory and the correct cost function is known as the **Kullback-Leibler divergence**:

\begin{equation}
\mathcal{C} = KL(p \| q) = \sum_{ij} p_{ij} \log \frac{p_{ij}}{q_{ij}} \,  .
\end{equation}

Now the only thing left to do is to choose the distributions $p_{ij}$ and $q_{ij}$.  The underlying heuristics for this are explained in the paper above, but a natural choice is a Gaussian distribution in the higher-dimensional space $D$:

\begin{equation}
p_{j \mid i}=\frac{\exp \left(-\left\|\boldsymbol{x}_{i}-\boldsymbol{x}_{j}\right\|^{2} / 2 \sigma_{i}^{2}\right)}{\sum_{k \neq i} \exp \left(-\left\|\boldsymbol{x}_{i}-\boldsymbol{x}_{k}\right\|^{2} / 2 \sigma_{i}^{2}\right)}
\end{equation}

where we interpret $p_{j \mid i}$ as the liklihood that point $\boldsymbol{x}_j$ is a neighbor of $\boldsymbol{x}_i$ and thus $p_{i \mid i} = 0$  These are high-dimensional Gaussian distributions and $\sigma_i$ are parameters that can be set by fixing the entropy of each data point:

\begin{equation}
H(p_i) \equiv - \sum_j p_{j \mid i} \log_2 p_{i \mid j}
\end{equation}

such that $\Upsilon = 2^{H(p_i)}$ is equal to the same number (the *perplexity*) for each data point.  This uniquely determines $\sigma_i$ for each point and ensures that low-density regions have larger $\sigma_i$ values.  To ensure that each data point has a non-vanishing contribution to the cost function, we define the symmetric distribution:

\begin{equation}
p_{ij} = \frac{p_{j \mid i} + p_{i \mid j}}{2N}\, 
\end{equation}

which is our final probability.  

For our lower-dimensional latent space we choose coordinate $\boldsymbol{Y} = \{\boldsymbol{y}_i\}_{j=1}^{N}$ where $\boldsymbol{y}_i \in \mathbb{R}^M$ with $M \ll D$. We could also choose a Gaussian in lower dimensions, but it turns out this has a problem: **the exponential supression at long distances from the Gaussian distribution is too restrictive in the latent space**.  As we don't have as much *space* here, it makes sense to relax the cost for incorrect distances for points that aren't too close in the $D$-dimensional space as these points are not strongly correlated. We thus want to find a distrubition with fatter tails than the Gaussian: **Student's t-distribution**:

\begin{equation}
q_{i j}=\frac{\left(1+\left\|\boldsymbol{y}_{i}-\boldsymbol{y}_{j}\right\|^{2}\right)^{-1}}{\sum_{k \neq i}\left(1+\left\|\boldsymbol{y}_{k}-\boldsymbol{y}_{l}\right\|^{2}\right)^{-1}}\, .
\end{equation}

#### Procedure:

1. $ p_{ij}$ are fixed by the $D$-dimensional data set.  Compute for a given perplexity.
2. Guess an initial set $\{\boldsymbol{y}_j\}$ in the latent space defined by $M$.
3. Compute $q_{ij}$ and the cost $\mathcal{C}$ given by the KL divergence.
4. Perform an update via gradient descent $\boldsymbol{y}_i \to \boldsymbol{y}_i - \eta \boldsymbol{\nabla}_{\boldsymbol{y}_i} \mathcal{C}$.
5. Iterate steps 3-4 until convergence.

Code from https://lvdmaaten.github.io/tsne/ is included as function `ml4s.tsne`.  Let's see how it works!


<!--
x = np.array([[0,0],[1,1],[4,4]])
y = np.array([0,1,50])

def p(i,j,x):
    σ = 1.0
    pji = np.exp(-np.sum((x[i]-x[j])**2)/(2*σ**2))
    norm = 0.0
    for k in range(x.shape[0]):
        if k != i:
            norm += np.exp(-np.sum((x[i]-x[k])**2)/(2*σ**2))
    pji /= norm
    
    pij = np.exp(-np.sum((x[j]-x[i])**2)/(2*σ**2))
    norm = 0.0
    for k in range(x.shape[0]):
        if k != j:
            norm += np.exp(-np.sum((x[j]-x[k])**2)/(2*σ**2))
    pij /= norm

    return (pij + pji) / (2*x.shape[0])

def q(i,j,y):
    qij = 1.0 / (1 + np.sum((y[i]-y[j])**2))
    norm = 0.0
    for k in range(y.shape[0]):
        if k != i:
            norm += 1.0 / (1 + np.sum((y[i]-y[k])**2))
    return qij/norm
-->

In [ ]:
# produce a D-dimensional data set of Gaussian point clouds
D = 100

# number of clouds
n_clouds = 10 

# number of points per cloud
N_cloud_points = 40 

# total number of pouints
N = n_clouds*N_cloud_points # total number of points

# cloud size
σ = 1.0

# create the clusters and labels 
# they are each distributed about a common point
X,y = np.zeros([N,D]), np.zeros([N])
for j in range(n_clouds):
    
    # cluster label
    y[j*N_cloud_points:(j+1)*N_cloud_points] = j
    
    # the center position of the cloud
    X̄ = np.random.normal(size=(D)) 
    
    # the cloud points
    X[j*N_cloud_points:(j+1)*N_cloud_points,:] = X̄[np.newaxis,:] + σ*np.random.normal(size=(N_cloud_points,D))

# Project clouds into 2D and color
# compare with PCA

fig,ax = plt.subplots(1,2, figsize=(10,4))
scatter = ax[0].scatter(X[:,0],X[:,1],c=y,cmap='Spectral_r')

legend = ax[0].legend(*scatter.legend_elements(),loc=(2.2,0), title="Cloud Label")
ax[0].add_artist(legend)
ax[0].set_xlabel(r'$X_1$')
ax[0].set_ylabel(r'$X_2$');

from sklearn.decomposition import PCA
M = 2
model = PCA(n_components=2)
XPCA = model.fit_transform(X)

# store the results
λ = model.explained_variance_
PCAj = model.explained_variance_ratio_
V = model.components_

pX = X @ V.T

scatter = ax[1].scatter(pX[:,0],pX[:,1],c=y,cmap='Spectral_r')
ax[1].set_xlabel(f'PCA-1 = {PCAj[0]:.2f}')
ax[1].set_ylabel(f'PCA-2 = {PCAj[1]:.2f}')
fig.subplots_adjust(hspace = 2)

### Apply the t-SNE Algorithm

In [ ]:
Y = ml4s.tsne(X, no_dims=2, initial_dims=50, perplexity=5.0, 
       do_animation=True, animation_skip_steps=10, max_iter=300)

fig,ax = plt.subplots()
scatter = ax.scatter(Y[:,0],Y[:,1],c=y,cmap='Spectral_r')

legend = ax.legend(*scatter.legend_elements(),loc=(1,0), title="Cloud Label")
ax.add_artist(legend)
ax.set_xlabel(r'$Y_1$')
ax.set_ylabel(r'$Y_2$');

### Do this for the MNIST Data Set

We project onto the first 50 princpal components. See [here](https://stats.stackexchange.com/questions/134282/relationship-between-svd-and-pca-how-to-use-svd-to-perform-pca) to understand the relationship between PCA and the singular value decomposition.

In [ ]:
from sklearn.datasets import fetch_openml

# MNIST exists in many places!
mnist = fetch_openml('mnist_784',parser='auto')
X = mnist.data
y = mnist.target.astype('int')

# Do PCA and project onto 50 dimensions
X = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X, full_matrices=False)
X50 = np.dot(U, np.diag(s))[:,:50]

#### t-SNE is Slow!

The conventional t-SNE algorithm we used above scales as $\mathrm{O}(N^2)$ where $N$ is the number of data points.  Recent algorithms use the [Fast Fourier transform](https://github.com/KlugerLab/FIt-SNE) and are extermely effecient. We load a cython wrapper around a c++ implementation https://github.com/KlugerLab/pyFIt-SNE.

In [ ]:
import fitsne

In [ ]:
%%time

# make sure the numpy array is prepared in C-order
X50 = X50.copy(order='C')

np.random.seed(0)
Y = fitsne.FItSNE(X50)

In [ ]:
%config InlineBackend.figure_format = 'retina'

fig,ax = plt.subplots()
scatter = ax.scatter(Y[:,0],Y[:,1],s=2,c=y,cmap='Spectral_r',edgecolors='None')

# produce a legend with the unique colors corresponding to digits
legend = ax.legend(*scatter.legend_elements(),loc=(1,0), title="MNIST Digits")
ax.add_artist(legend)

## Clustering Algorithms

In general there is a zoo of clustering algorithms that can discover structure in unlabelled data.  While they can certainly aid in data visualization, the goal of clutering is to group unlabelled data into distinct classes (clusters) based on some similarity or distance measure.  We usually take this to be the euclidean distance.  

Different algorithms have different tradeoffs depending on the type of data under consideration (e.g. overlapping vs. well separated, low vs. high-dimensional data) and have different algorithmic scaling.  Many are included in `sklearn` and here we take the example code from  [here](https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#sphx-glr-auto-examples-cluster-plot-cluster-comparison-py) to get a flavor for how they work on some toy datasets.

In [ ]:
%%time
%config InlineBackend.figure_format = 'retina'

import time
import warnings

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

# ============
# Generate datasets. We choose the size big enough to see the scalability
# of the algorithms, but not too big to avoid too long running times
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Anisotropicly distributed data
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Set up cluster parameters
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # update parameters with dataset-specific values
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # normalize dataset for easier parameter selection
    X = StandardScaler().fit_transform(X)

    # estimate bandwidth for mean shift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # connectivity matrix for structured Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # make connectivity symmetric
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Create cluster objects
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('AgglomerativeClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)
        
        _colors = np.array(colors[::2])
        # add black color for outliers (if any)
        _colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=_colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()